## This notebook runs a simulation of a rope using ANCF elements to model the flexibility and the DCA to form and solve the equations of motion

In [1]:
import math
import pickle 
import numpy as np
import scipy as sp
import sympy as sym
import matplotlib.pyplot as plt
import MBstructs as MB
import MultiBodyFuncts as MBF
import dca_recursive
import contextlib

from numpy.linalg import inv
from scipy.integrate import odeint

%matplotlib inline

IndentationError: expected an indented block (MBstructs.py, line 60)

In [ ]:
@contextlib.contextmanager
def printoptions(*args, **kwargs):
    original = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    yield 
    np.set_printoptions(**original)

### System Variables

In [2]:
# number of bodies and GEBF elements
nGEBF = 12
n = nGEBF
# number of degrees of freedom for the 
# 3D-GEBF element with a single planar rotation
ndofsGEBF = 8
ndofsKIN  = 1 # (one pin joint)

### Initial Conditions

In [3]:
# start the rope from the horizontal 
# Compute initial generalized coordinates
# q = ['u_1x','u_1y',theta_1,'u_2x','u_2y','theta_2']
# u = q_dot
# state = [q1 u1 q2 u2 ... qn un]
q0GEBF = [np.zeros((1,8)) for i in range(1,n+1)]
u0GEBF = [np.zeros((1,8)) for i in range(1,n+1)]
stateGEBF = np.array([np.hstack((q0,u0)).reshape(1,2*ndofsGEBF) 
                      for q0,u0 in zip(q0GEBF,u0GEBF)]).reshape(nGEBF,2*ndofsGEBF)
q0KIN = [0]
u0KIN = [0]

stateRIGID = np.array([np.hstack((q0,u0)).reshape(1,2*ndofsKIN) 
                      for q0,u0 in zip(q0KIN,u0KIN)]).reshape(ndofsKIN,2*ndofsKIN)
# state = stateGEBF

### Initilize the bodies and joints of the system (compute inertial properties)
12 GEBF bodies hooked to the ground

In [4]:
# Create a list of Joints
jointsKIN = [MB.Joint(np.array([0,0,1,0,0,0]),1)]
jointsGEBF = [MB.Joint(np.zeros((1,6)),0) for i in range (n)]
joints = jointsKIN + jointsGEBF
# GEBF-bodies do not need to be initialized 
# Rigid-bodies do not need to be initialized

NameError: name 'MB' is not defined

In [6]:
# For systems with bodies of different type:
#   bodies are numbered from left to right

# initialize(bodies, y, n) (rigid bodies)

# GEBF-Properties
# Physical Properties
A   =  0.0018
I   =  1.215e-8
L   =  1.2
r   =  math.sqrt(A/math.pi)
l   =  L/n

# Material Properties
E   =  0.7e6
rho =  5540

# update state depedent quantities (all quantities for GEBF so just re-initialize)
bodiesGEBF = [MB.GEBF_Element(E, I, r, rho, l, state)  for state in stateGEBF]
bodies = bodiesGEBF

# DCA returns the eddot vector
sol = dca_recursive.solve(n,0,bodies,joints,2,1)

# uKIN = np.dot(np.transpose(joints[0].P*sol.pop(0)))
# uGEBF = [sol.pop(0)[2:6].reshape(1,4) for i in range(5)]

LinAlgError: Singular matrix

In [7]:
# [joints[i].D for i in range(len(bodies))]
joints[1]

In [6]:
def simulate(stateGEBF, stateKIN, t, n, bodies, joints, BC1, BC2):
    """
    1) This fuction updates the internal forces before calling DCA
    2) This function comptues the generalized cooridnates after the 
    accelerations and constraint forces are determined by the 
    recursive (function) DCA call
    """ 
    # For systems with bodies of different type:
    #   bodies are numbered from left to right
    
    # initialize(bodies, y, n) (rigid bodies)
    
    # GEBF-Properties
    # Physical Properties
    A   =  0.0018
    I   =  1.215e-8
    L   =  1.2
    r   =  math.sqrt(A/math.pi)
    l   =  L/n

    # Material Properties
    E   =  0.7e6
    rho =  5540

    # update state depedent quantities (all quantities for GEBF so just re-initialize)
    bodiesGEBF = [MB.GEBF_Element(E, I, r, rho, l, state)  for state in stateGEBF]     

    # DCA returns the eddot vector
    sol = dca_recursive(n,0,bodies,joints,BC1,BC2)
    
    uKIN = np.dot(np.transpose(joints[0].P*sol.pop(0)))
    uGEBF = [sol.pop(0)[2:6].reshape(1,4) for i in range(5)]
    
#     ydot=np.zeros((1,2*n))
#     for j in range(0,n):
#         if j == 0:
#             A1 = sol.pop(0)
#             ydot[j+n]=np.dot(np.transpose(joints[j].P),A1)
#         else:
#             A2 = sol.pop(0)
#             A1 = sol.pop(0)
#             ydot[j+n]=np.dot(np.transpose(joints[j].P),(A1-A2))
    
#     q0GEBF = [np.zeros((1,8)) for i in range(1,n+1)]
#     u0GEBF = [np.zeros((1,8)) for i in range(1,n+1)]
#     stateGEBF = np.array([np.hstack((q0,u0)).reshape(1,2*ndofsGEBF) 
#                           for q0,u0 in zip(q0GEBF,u0GEBF)]).reshape(nGEBF,2*ndofsGEBF)
#     q0KIN = [0]
#     u0KIN = [0]

#     stateRIGID = np.array([np.hstack((q0,u0)).reshape(1,2*ndofsKIN) 
#                           for q0,u0 in zip(q0KIN,u0KIN)]).reshape(ndofsKIN,2*ndofsKIN)
#     # state = stateGEBF

    ydot[:n]=state[n:]
    return ydot 

### Integration

In [7]:
# Length of time of the simulation
Time=np.arange(0,5,.01)
# odeint is the numerical integrator used
x0 = state.reshape(1,size(state))
y=odeint(simulate,x0,Time,(n,0,bs,js,2,1))

NameError: name 'state' is not defined

In [ ]:
# Energy Calculation
#--------------------
#The energy of the system is calculated and plotted

energy=MBF.PendEnergy(y,elements)
KE=energy[:,0]
PE=energy[:,1]
TE=energy[:,2]

plt.plot(t,TE-TE[0])
plt.xlabel("Time [s]")
plt.ylabel("energy")
plt.title("System Energy")
plt.show()

plt.plot(t,PE,Time,KE)
plt.xlabel("Time[s]")
plt.ylabel("energy")
plt.title("Kinetic and Potential Energy")
plt.show

In [ ]:
# Solution Plot
#--------------------
plt.plot(Time,yy[:,:n])
plt.xlabel("Time [s]")
plt.ylabel("Generalized Coordinates [Rad]")
plt.title("System Response")

plt.show()

plt.plot(Time,yy[:,n:])
plt.xlabel(("Time[s]"))
plt.ylabel(("Generalized Speeds [Rad/s]"))
plt.title("System Response")

plt.show()

In [3]:
25%2

1

In [9]:
import math
print(math.trunc(40/2))
print(40//2)

20
20


In [32]:
test = [np.linspace(1,4,4).reshape(4,1) for i in range(5)]

In [33]:
test

[array([[ 1.],
        [ 2.],
        [ 3.],
        [ 4.]]), array([[ 1.],
        [ 2.],
        [ 3.],
        [ 4.]]), array([[ 1.],
        [ 2.],
        [ 3.],
        [ 4.]]), array([[ 1.],
        [ 2.],
        [ 3.],
        [ 4.]]), array([[ 1.],
        [ 2.],
        [ 3.],
        [ 4.]])]

In [34]:
[test.pop(0)[2:5].reshape(1,2) for i in range(5)]

[array([[ 3.,  4.]]),
 array([[ 3.,  4.]]),
 array([[ 3.,  4.]]),
 array([[ 3.,  4.]]),
 array([[ 3.,  4.]])]

In [5]:
l = 1
r = np.array([l/2,0,0])

In [14]:
np.hstack(([1,0,0],np.vstack(([r[1], r[0]],np.eye(2)))))

ValueError: all the input arrays must have same number of dimensions

In [9]:
np.array([r[1], r[0]])

array([ 0. ,  0.5])

In [17]:
np.hstack((np.array([[1],[0],[0]]),np.vstack(([r[1], r[0]],np.eye(2)))))

array([[ 1. ,  0. ,  0.5],
       [ 0. ,  1. ,  0. ],
       [ 0. ,  0. ,  1. ]])

In [27]:
I  =500
m = 23

In [28]:
np.hstack(((np.array([[I],[0],[0]]),np.vstack((np.zeros((1,2)),m*np.eye(2))))))

array([[ 500.,    0.,    0.],
       [   0.,   23.,    0.],
       [   0.,    0.,   23.]])